# Filtering rows 2: Using `filter` and the Expression API

By the end of this lecture you will be able to:
- apply conditions with the `filter` method
- add a row number column
- parition a `DataFrame`

The `filter` method is our first example of the *Expression API*.

_**Learning to use the *Expression API* is the most important step to writing high performance queries in Polars**_


In [1]:
import polars as pl

In [2]:
csv_file = "../data/titanic.csv"

In [3]:
df = pl.read_csv(csv_file)
df.head(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""


## Applying conditions with `filter`

We use the `filter` method to filter rows according to a condition.

> In Pandas we often use a boolean mask to filter rows but in Polars we use `filter`. Note also that the `filter` method in Polars is quite different from the filter method in Pandas.

We first use an *expression* in the `filter` method before we examine the syntax in more detail.

In this example we want to keep all rows with the first class passengers

In [4]:
(
    df
    .filter(
        pl.col('Pclass') == 1
    )
    .head(2)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""


## Syntax of `filter`
Inside the `filter` method we pass an _**expression**_ and apply a Boolean condition to it:

`pl.col('Pclass') == 1`

This expression has two parts:
- `pl.col('Pclass')` expression selects the `Pclass` column from `df`
- `== 1` applies a Boolean condition to this expression

In this example we choose all rows with the number of parents & children (`Parch`) is greater than 1

In [5]:
(
    df
    .filter(
        pl.col('Parch') > 1
    )
    .head(2)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
9,1,3,"""Johnson, Mrs. …","""female""",27.0,0,2,"""347742""",11.1333,null,"""S"""
14,0,3,"""Andersson, Mr.…","""male""",39.0,1,5,"""347082""",31.275,null,"""S"""


As well as the mathemtical operators such as `==`,`>`,`<` there are corresponding text operators that some people find more readable

In [6]:
(
    df
    .filter(
        pl.col('Parch').gt(1)
    )
    .select("PassengerId","Parch","SibSp")
    .head(5)
)

PassengerId,Parch,SibSp
i64,i64,i64
9,2,0
14,5,1
26,5,1
28,2,3
44,2,1


You can see the full set of operators here: https://pola-rs.github.io/polars/py-polars/html/reference/expressions/operators.html

We can make a filter condition based on two expressions (i.e. comparing data in one column to another) rather than one expression and a constant. In this example we find rows where the number of parents & children (`Parch`) is greater than the number of siblings (`SibSp`)

In [7]:
(
    df
    .filter(
        pl.col('Parch').gt(pl.col("SibSp"))
    )
    .select("PassengerId","Parch","SibSp")
    .head(5)
)

PassengerId,Parch,SibSp
i64,i64,i64
9,2,0
14,5,1
26,5,1
44,2,1
55,1,0


To save a bit of typing we can also apply a filter to a column by passing the column name directly

In [8]:
(
    df
    .filter(
        Parch = 3,
    )
    .select("PassengerId","Parch","SibSp")
    .head(5)
)

PassengerId,Parch,SibSp
i64,i64,i64
87,3,1
438,3,2
737,3,1
775,3,1
859,3,0


This approach only works for equality conditions (i.e. not for >,< etc). 

Why does this simple approach only work for equalities? Because in this approach Polars takes advantage of Python keyword arguments - we are basically "pretending" we are calling `filter` with an argument called `Parch` equal to 3 which Polars internally converts to `pl.col("Parch") == 3`. Python only lets us use this trick with the `=` operator

### Conditions based on row numbers with `filter`

We can add an explicit row number column using `with_row_index` on a `DataFrame`

In [9]:
df = pl.read_csv(csv_file)
df = df.with_row_index(name='index')
df.head(3)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
0,1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
2,3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""


We can then use `filter` to apply a condition based on row number

In [10]:
(
    df
    .filter(
        pl.col('index') < 4
    )
)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
0,1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
2,3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""


However, a simpler way to do this is with `slice`

In [11]:
(
    df
    .slice(0,4)
)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
0,1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
2,3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""


### Filtering on a Boolean column
We can filter for `True` values on a Boolean column by passing the column as an expression to `filter` without a condition

In [12]:
(
    df
    .with_columns(
        less_than_30 = pl.col("Age") < 30
    )
    .filter(
        pl.col("less_than_30")
    )
    .head(2)
)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,less_than_30
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,bool
0,1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""",true
2,3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S""",true


We can negate a filter condition with `~`

In [13]:
(
    df
    .with_columns(
        less_than_30 = pl.col("Age") < 30
    )
    .filter(
        ~pl.col("less_than_30")
    )
    .head(2)
)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,less_than_30
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,bool
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""",false
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""",false


or with the `not_` expression

In [14]:
(
    df
    .with_columns(
        less_than_30 = pl.col("Age") < 30
    )
    .filter(
        pl.col("less_than_30").not_()
    )
    .head(2)
)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,less_than_30
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,bool
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""",false
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""",false


## Partitioning a `DataFrame`
In some cases we want to get the different subsets of the `DataFrame` that result from a single condition. 

We can do this partition into sub-`DataFrames` with the `partition_by` method.

In this example we partition by the `Pclass` column

In [15]:
df_pclass_dict = (
    df
    .partition_by(by=["Pclass"],as_dict=True)
)

The output is a python `dict` mapping from the unique values in `Pclass` to the sub-`DataFrame` for each class. This partition requires copying the data in `df` to new sub-`DataFrames`.

Note that the keys of this `dict` are always tuples even if there is just one element in the tuple for each key

In [16]:
df_pclass_dict.keys()

dict_keys([(3,), (1,), (2,)])

Note that if we don't pass the `as_dict=True` argument we instead get a python `list` of sub-`DataFrames`.

We can get the rows with first class passengers from this `dict` (note the `,` which turns `1` into the tuple `(1,)`

In [17]:
df_pclass_dict[1,].head(2)

index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""


## Filter in lazy mode
We create a `LazyFrame` by scanning the CSV and adding a `filter` operation

In [18]:
(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age") > 30)
)

When we print the optimized plan we see the `filter` operation is part of the `SELECTION`. This query optimisation is called **predicate pushdown**. With predicate pushdown Polars tries to apply a `filter` as early as possible in a query plan to reduce the amount of data that must be processed

In [19]:
print(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age") > 30)
    .explain()
)


  Csv SCAN ../data/titanic.csv
  PROJECT */12 COLUMNS
  SELECTION: [(col("Age")) > (30.0)]


In this case of a `filter` applied on a query like this from a CSV on our local machine the query optimisation will not have much impact: Polars just reads the CSV, makes a `DataFrame` in memory and then filters the `DataFrame`. The result would probably be similar to doing the query in eager mode.

However, if we are reading a file from cloud storage then Polars tries to apply the condition in `SELECTION` in the cloud storage and so reduces the amount of data that must be transferred across the network. The transfer across the network is typically the slowest and most expensive part of the query.



If we set `streaming=True` in `explain` we see that the `filter` operation is inside the 
```
--- STREAMING
--- END STREAMING
```
part of the query plan - this means that Polars can do this filter operation in streaming mode if we evaluate the lazy query with `.collect(streaming=True)`

In [20]:
print(
    pl.scan_csv(csv_file)
    .filter(pl.col("Age") > 30)
    .explain(streaming=True)
)

--- STREAMING

  Csv SCAN ../data/titanic.csv
  PROJECT */12 COLUMNS
  SELECTION: [(col("Age")) > (30.0)]  --- END STREAMING

  DF []; PROJECT */0 COLUMNS; SELECTION: "None"


C:\Users\renan\AppData\Local\Temp\ipykernel_17392\4261519831.py:4: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  .explain(streaming=True)


# Exercises
In the exercises you will develop your understanding of
- using the `filter` method
- adding a row number column
- partitioning a `DataFrame`

## Solutions

### Solution to Exercise 1
Select all rows with `Age` greater than 30

In [23]:
(
    pl.read_csv(csv_file)
    .filter(pl.col('Age') > 30)
    .head(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


Select all rows where `Embarked` is equal to "C" - using the keyword approach

In [24]:
(
    pl.read_csv(csv_file)
    .filter(Embarked = "C")
    .head(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
10,1,2,"""Nasser, Mrs. N…","""female""",14.0,1,0,"""237736""",30.0708,null,"""C"""
20,1,3,"""Masselmani, Mr…","""female""",null,0,0,"""2649""",7.225,null,"""C"""


Select all rows where `Embarked` is equal to "C" - use `pl.col` with the text operator rather than the mathematical operator this time

In [25]:
(
    pl.read_csv(csv_file)
    .filter(pl.col("Embarked").eq("C"))
    .head(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
10,1,2,"""Nasser, Mrs. N…","""female""",14.0,1,0,"""237736""",30.0708,null,"""C"""
20,1,3,"""Masselmani, Mr…","""female""",null,0,0,"""2649""",7.225,null,"""C"""


Select all rows where `Embarked` is **not** equal to "C" 

In [26]:
(
    pl.read_csv(csv_file)
    .filter(~pl.col("Embarked").eq("C"))
    .head(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""


### Solution to Exercise 2
Add a row number column

In [27]:
(
    pl.read_csv(csv_file)
    .with_row_index("row_nr")
)

row_nr,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
0,1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
2,3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
4,5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…,…
886,887,0,2,"""Montvila, Rev.…","""male""",27.0,0,0,"""211536""",13.0,null,"""S"""
887,888,1,1,"""Graham, Miss. …","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S"""
888,889,0,3,"""Johnston, Miss…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S"""


Continue by selecting the first 5 rows using `filter` on the row number column

In [28]:
(
    pl.read_csv(csv_file)
    .with_row_index("row_nr")
    .filter(pl.col("row_nr")<5)
)

row_nr,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
0,1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
1,2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
2,3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""
3,4,1,1,"""Futrelle, Mrs.…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
4,5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


### Solution to Exercise 3
Partition the `DataFrame` by the `Survived` and `Pclass` columns as a `dict`

In [29]:
survived_pclass_dict = (
    pl.read_csv(csv_file)
    .partition_by("Survived","Pclass",as_dict=True)
)

In [30]:
survived_pclass_dict.keys()

dict_keys([(0, 3), (1, 1), (1, 3), (0, 1), (1, 2), (0, 2)])

Return the sub-`DataFrame` with the passengers who did not survive from the third class

In [31]:
(
    survived_pclass_dict[(0,3)]
    .head(2)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
5,0,3,"""Allen, Mr. Wil…","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


### Solution to Exercise 4
In this exercise we load data from the Spotify charts in a compressed CSV

In [32]:
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"
spotify_df = pl.read_csv(spotify_csv)
spotify_df.head()

title,rank,date,artist,url,region,chart,trend,streams
str,i64,str,str,str,str,str,str,i64
"""Starboy""",1,"""2017-01-01""","""The Weeknd, Da…","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",3135625
"""Closer""",2,"""2017-01-01""","""The Chainsmoke…","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",3015525
"""Let Me Love Yo…",3,"""2017-01-01""","""DJ Snake, Just…","""https://open.s…","""Global""","""top200""","""MOVE_UP""",2545384
"""Rockabye (feat…",4,"""2017-01-01""","""Clean Bandit""","""https://open.s…","""Global""","""top200""","""MOVE_DOWN""",2356604
"""One Dance""",5,"""2017-01-01""","""Drake, WizKid,…","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",2259887


Filter the `DataFrame` to find all rows with artist Post Malone

In [33]:
(
    spotify_df
    .filter(
        pl.col("artist") == "Post Malone"
    )
)

title,rank,date,artist,url,region,chart,trend,streams
str,i64,str,str,str,str,str,str,i64
"""White Iverson""",196,"""2017-01-01""","""Post Malone""","""https://open.s…","""Global""","""top200""","""NEW_ENTRY""",332756
"""White Iverson""",188,"""2017-01-02""","""Post Malone""","""https://open.s…","""Global""","""top200""","""MOVE_UP""",343936
"""Psycho (feat. …",2,"""2018-03-01""","""Post Malone""","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",5999224
"""I Fall Apart""",22,"""2018-03-01""","""Post Malone""","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",2003396
"""Candy Paint""",64,"""2018-03-01""","""Post Malone""","""https://open.s…","""Global""","""top200""","""MOVE_UP""",1065141
…,…,…,…,…,…,…,…,…
"""White Iverson""",184,"""2018-01-30""","""Post Malone""","""https://open.s…","""Global""","""top200""","""MOVE_UP""",590653
"""I Fall Apart""",18,"""2018-01-31""","""Post Malone""","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",2034143
"""Candy Paint""",55,"""2018-01-31""","""Post Malone""","""https://open.s…","""Global""","""top200""","""SAME_POSITION""",1247324
